# ASSIGNMENT 1 - EXCERCISE 1

### Determine how much of a product should be produced to maximize the profit

In [1]:
#Import library
from pulp import *

In [2]:
#Definition of suppliers
#In this case, suppliers are the three machines
#Milling machine: A, Lathe: B, Grinder: C
Machines = ["A","B","C"]

In [3]:
#Definition of the variable to be maximized
#In this case, products are the variables to be maximized
Products = ["1","2","3"]

In [4]:
#Definition of supply of products per machine
supply = {"A":500,"B":350, "C":150}

In [5]:
#Definition of profits per product to maximize
profits = [[50],[20],[25]]

In [6]:
#Definition of demand
#In this case, the demand is the sales potential per product, per week
#Demand of product 1 is that of maximum possible production when all machines are used together for it: 500/9 + 350/5 + 150/3
#Demand of product 2 is that of maximum possible production when all machines are used together for it: 500/3 + 350/4
demand = {"1":175,"2":254,"3":20}

In [7]:
#Associate profits list to each product
profits = makeDict([Machines,Products],profits,0)

In [8]:
#Instruction for maximizing the linear arrangement
prob = LpProblem("Production_Problem",LpMaximize)

In [9]:
#Create combinations of type of product per machine
Production = [(m,p) for m in Machines for p in Products]
print (Production)

[('A', '1'), ('A', '2'), ('A', '3'), ('B', '1'), ('B', '2'), ('B', '3'), ('C', '1'), ('C', '2'), ('C', '3')]


In [10]:
#Create a dictionary to associate the products made per each machine
#0 is the lowest limit of products made per machine and all results are integers
vars = LpVariable.dicts("Production",(Machines,Products),0,None,LpInteger)
print (vars)

{'A': {'1': Production_A_1, '2': Production_A_2, '3': Production_A_3}, 'B': {'1': Production_B_1, '2': Production_B_2, '3': Production_B_3}, 'C': {'1': Production_C_1, '2': Production_C_2, '3': Production_C_3}}


In [11]:
#Create the objective function by adding the profit per product
prob += lpSum([vars[m][p]*profits[m][p] for (m,p) in Production]),"Sum_of_Profits"

In [12]:
#Formulation of the problem: objective function, constraints, definition of variables
for m in Machines:
    prob += lpSum([vars[m][p] for p in Products])<=supply[m],"Sum_of_Products_out_of_Machine_%s"%m
for p in Products:
    prob += lpSum([vars[m][p] for m in Machines])>=demand[p],"Sum_of_Sold_Products%s"%p
print (prob)

Production_Problem:
MAXIMIZE
50*Production_A_1 + 20*Production_B_1 + 25*Production_C_1 + 0
SUBJECT TO
Sum_of_Products_out_of_Machine_A: Production_A_1 + Production_A_2
 + Production_A_3 <= 500

Sum_of_Products_out_of_Machine_B: Production_B_1 + Production_B_2
 + Production_B_3 <= 350

Sum_of_Products_out_of_Machine_C: Production_C_1 + Production_C_2
 + Production_C_3 <= 150

Sum_of_Sold_Products1: Production_A_1 + Production_B_1 + Production_C_1 >= 175

Sum_of_Sold_Products2: Production_A_2 + Production_B_2 + Production_C_2 >= 254

Sum_of_Sold_Products3: Production_A_3 + Production_B_3 + Production_C_3 >= 20

VARIABLES
0 <= Production_A_1 Integer
0 <= Production_A_2 Integer
0 <= Production_A_3 Integer
0 <= Production_B_1 Integer
0 <= Production_B_2 Integer
0 <= Production_B_3 Integer
0 <= Production_C_1 Integer
0 <= Production_C_2 Integer
0 <= Production_C_3 Integer



In [13]:
#Solve the optimization problem
#Find the number of products per machine, the overall number of products and the total profit
prob.solve()
print("Status:",LpStatus[prob.status])
Product_1 = 0
Product_2 = 0
Product_3 = 0
for i in range(len(Products)):
    Product_1 += prob.variables()[i*3].varValue
    Product_2 += prob.variables()[i*3+1].varValue
    Product_3 += prob.variables()[i*3+2].varValue
print("Number of products 1 = ",Product_1)
print("Number of products 2 = ",Product_2)
print("Number of products 3 = ",Product_3)
print("Total Profits per week = $",value(prob.objective))

Status: Optimal
Number of products 1 =  726.0
Number of products 2 =  254.0
Number of products 3 =  20.0
Total Profits per week = $ 30270.0


In [15]:
#To know in detail from which machine each product was made, the following code shows the result of each combination pair
#of machine-product
for v in prob.variables():
    print(v.name, "=", v.varValue)

Production_A_1 = 500.0
Production_A_2 = 0.0
Production_A_3 = 0.0
Production_B_1 = 76.0
Production_B_2 = 254.0
Production_B_3 = 20.0
Production_C_1 = 150.0
Production_C_2 = 0.0
Production_C_3 = 0.0


In [16]:
#We conclude that product 1 was made in all machines. Machine A produced 500 units, machine B produced 76 units and machine C
#produced 150 units.
#Product 2 was made solely in machine B and product 3 only in machine C. 